# Draft: Use of CGC w/ open data cube

This note book contains draft code to connect to an existing ODC instance, retrieve data, and luster it using CGC

Import path and add full local directory tree

In [ ]:
from sys import path
path.append("../")

## Loading data

### Specify ROI
Specify the ROI in terms of (rectagula) lat,lon bounds

In [ ]:
lon = (lon1, lon2)
lat = lat1, lat2


### Pass ROI settings to display


In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = lat, longitude = lon)

### data selection
Specify platform (i.e. observatory/data source) product type and define data selection

specification of measurments is opttional

In [ ]:
platform        = 'plaatform' # e.g. LANDAT_7
product_type    = 'dataprod' # e.g. ls7_ledaps_togo

from datetime import datetime 

params = dict(platform=platform,
              product=product_type,
              time=(datetime(2007,1,1), datetime(2007,12,31)) ,
              lon= lon,
              lat= lat,
              measurements = ['mes1', 'mes2', 'mes3'] )



### Create data cube object
In essence this means connectiing o an existing data cube. One should specify a name for he instance (app). The config file determines, a.o., which datacube one connects to.

In [ ]:
import datacube
dc = datacube.Datacube(app= "some_app_name", config='/home/localuser/.datacube.conf')

### Load data
Here data is loaded using the parameters specified above. It is also, of course, possible to define these explicitly in the load statement.

`data` is an `xarray` `Dataset` object and should have the coordinates `time`, `latitude`, and `longitude`. All specifie measurements should be present as variables, with commensurate dimensionality of the `xarray.Dataset` object


data = dc.load(**params)

## NB
The above is applicable to a situation in which extent, and desired data/measurements are known. If this is not the case, and availble data needs to be explored the procedure below enables a more interactive exploratory loading 

### Browse available products in data cube

In [ ]:
list_of_products = dc.list_of_products()




Use data acces API to get extents for a chosen product (chosen, e.g. from list of products)

In [ ]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "PLATFORM"
product = "PRODUCT"

coordinates = api.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = (min(coordinates['latitude'].values),max(coordinates['latitude'].values))
print( latitude_extents )

longitude_extents = (min(coordinates['longitude'].values),max(coordinates['longitude'].values))
print( longitude_extents )

time_extents = (min(coordinates['time'].values),max(coordinates['time'].values))
print( time_extents )



#### Pick coordinate extents and measurements to load and then use `dc.load` as above

# Interface with CGC

`CGC` requires the reshaping of the data from 3 to 2 dimensions collpasing the spatial coordinates. Here we suggest using the 
`xarray.Dataset.to_stacked_array()` method along the lines of

`data.to_stacked_array('spatialstack',"time",variable_dim='latlon')`